In [1]:
import pyblp
import numpy as np
import pandas as pd
import statsmodels.api as sm

pyblp.options.digits = 2
pyblp.options.verbose = False
pyblp.__version__

# Load the data 
sales_data = pd.read_csv("OTC_Sales.csv", header=0, sep='\t')
print(sales_data.head())

demo_data = pd.read_csv("OTC_Demographics.csv", header=1)
print(demo_data.head())

   store  week  brand  sales_  count  price_  prom_  cost_
0      2     1      1      16  14181    3.29    0.0   2.06
1      2     2      1      12  13965    3.27    0.0   2.04
2      2     3      1       6  13538    3.37    0.0   2.15
3      2     4      1      12  13735    3.30    0.0   2.07
4      2     5      1      10  13735    3.34    0.0   2.12
   store      educ     income
0      2  0.248935  10.553210
1      5  0.321226  10.922370
2      8  0.095173  10.597010
3      9  0.222172  10.787150
4     12  0.253413   9.996659


In [2]:
#QUESTION 1
#Now, well start the calculations for the Sum Stats Table. 
#Market Shares 
#Note- for this table I want the market shares to be meaningful in terms of one another (headache medicine). 
# total sales per store-week 
sales_data["total_sales"] = sales_data.groupby(["store", "week"])["sales_"].transform("sum")

# market share: sales of product i / total sales in the store-week
sales_data["market_share"] = sales_data["sales_"] / sales_data["total_sales"]

# Calculate market share for each product (brand)
market_share = sales_data.groupby("brand")["market_share"].mean()

# Print the market share for each product
for brand, share in market_share.items():
    print(f"Market Share of Product_{brand}: {share:.2%}")


Market Share of Product_1: 14.88%
Market Share of Product_2: 17.76%
Market Share of Product_3: 11.65%
Market Share of Product_4: 12.01%
Market Share of Product_5: 7.71%
Market Share of Product_6: 3.60%
Market Share of Product_7: 4.33%
Market Share of Product_8: 3.47%
Market Share of Product_9: 8.16%
Market Share of Product_10: 9.30%
Market Share of Product_11: 7.13%


In [3]:
#Unit Price (price per 1 bottle of a product)
unit_price = sales_data.groupby("brand")["price_"].mean()

# average unit price for each product 
for product, price in unit_price.items():
    print(f"Average Unit Price of Product_{product}: {price:.2f}")


Average Unit Price of Product_1: 3.42
Average Unit Price of Product_2: 4.94
Average Unit Price of Product_3: 7.02
Average Unit Price of Product_4: 2.96
Average Unit Price of Product_5: 5.15
Average Unit Price of Product_6: 8.16
Average Unit Price of Product_7: 2.67
Average Unit Price of Product_8: 3.61
Average Unit Price of Product_9: 3.97
Average Unit Price of Product_10: 1.93
Average Unit Price of Product_11: 4.45


In [4]:
#price per 100 tablets 
price_per_100 = {}

# Tablets per bottle for each product 
tablets_per_bottle = {
    1: 25,  # Tylenol 25
    2: 50,  # Tylenol 50
    3: 100,  # Tylenol 100
    4: 25,  # Advil 25
    5: 50,  # Advil 50
    6: 100,  # Advil 100
    7: 25,  # Bayer 25
    8: 50,  # Bayer 50
    9: 100,  # Bayer 100
    10: 50,  # Store Brand 50
    11: 100  # Store Brand 100
}

# Calculate price per 100 tablets based on the number of tablets per bottle
for product, price in unit_price.items():
    if tablets_per_bottle[product] == 25:
        price_per_100[product] = price * 4  
    elif tablets_per_bottle[product] == 50:
        price_per_100[product] = price * 2  
    elif tablets_per_bottle[product] == 100:
        price_per_100[product] = price  

for product, price in price_per_100.items():
    print(f"Price per 100 tablets of Product_{product}: {price:.2f}")


Price per 100 tablets of Product_1: 13.68
Price per 100 tablets of Product_2: 9.88
Price per 100 tablets of Product_3: 7.02
Price per 100 tablets of Product_4: 11.85
Price per 100 tablets of Product_5: 10.29
Price per 100 tablets of Product_6: 8.16
Price per 100 tablets of Product_7: 10.69
Price per 100 tablets of Product_8: 7.21
Price per 100 tablets of Product_9: 3.97
Price per 100 tablets of Product_10: 3.86
Price per 100 tablets of Product_11: 4.45


In [5]:
#unit wholesale price (this will be the avg cost of each product)
unit_wholesale_price = sales_data.groupby("brand")["cost_"].mean()

for product, cost in unit_wholesale_price.items():
    print(f"Average Wholesale Price of Product_{product}: {cost:.2f}")


Average Wholesale Price of Product_1: 2.18
Average Wholesale Price of Product_2: 3.67
Average Wholesale Price of Product_3: 5.75
Average Wholesale Price of Product_4: 2.03
Average Wholesale Price of Product_5: 3.62
Average Wholesale Price of Product_6: 6.09
Average Wholesale Price of Product_7: 1.85
Average Wholesale Price of Product_8: 2.42
Average Wholesale Price of Product_9: 3.71
Average Wholesale Price of Product_10: 0.91
Average Wholesale Price of Product_11: 1.92


In [6]:
# QUESTION 2
# part a
# To calculate market share Sij, im going to calculate the total market size. 
# Sij will be the number who buy product i in market j /market size (count)
sales_data["market_share"] = sales_data["sales_"] / sales_data["count"]

# outside option share S0j = (any buyer - buyers of headache medicine) / market size
sales_data["outside_share"] = 1 - sales_data.groupby(["store", "week"])["market_share"].transform("sum")

# log market shares (log(s_ij) - log(s_0j))
sales_data["logit_share"] = np.log(sales_data["market_share"]) - np.log(sales_data["outside_share"])

#i want to do this ols regression using the statsmodels 
formula = "logit_share ~ 1+ price_ + prom_"
model = sm.OLS.from_formula(formula, data=sales_data)
print(model.fit().summary())


                            OLS Regression Results                            
Dep. Variable:            logit_share   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     315.2
Date:                Tue, 11 Feb 2025   Prob (F-statistic):          1.72e-136
Time:                        18:04:10   Log-Likelihood:                -50711.
No. Observations:               38544   AIC:                         1.014e+05
Df Residuals:                   38541   BIC:                         1.015e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -7.7660      0.012   -627.345      0.0

In [7]:
# Convert 'product' to numeric (if it's meant to be 1-11)
sales_data["brand"] = pd.to_numeric(sales_data["brand"], errors="coerce")
# Check if the conversion worked
print(sales_data["brand"].dtype)  # Should now be int64 or float64
print(sales_data.dtypes)


int64
store              int64
week               int64
brand              int64
sales_             int64
count              int64
price_           float64
prom_            float64
cost_            float64
total_sales        int64
market_share     float64
outside_share    float64
logit_share      float64
dtype: object


In [ ]:
# question 2 part b
# ols regression with price and promo as product characteristics and product fixed effects
formula = "logit_share ~ 1 + price_ + prom_ + C(brand)"
model = sm.OLS.from_formula(formula, data=sales_data)
print(model.fit().summary())

                            OLS Regression Results                            
Dep. Variable:            logit_share   R-squared:                       0.460
Model:                            OLS   Adj. R-squared:                  0.460
Method:                 Least Squares   F-statistic:                     2733.
Date:                Tue, 11 Feb 2025   Prob (F-statistic):               0.00
Time:                        18:04:10   Log-Likelihood:                -39156.
No. Observations:               38544   AIC:                         7.834e+04
Df Residuals:                   38531   BIC:                         7.845e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -6.0709      0.036   -166.

In [ ]:
# question 2 part c 
# ols regression with price and promo as product characteristics and product-store fixed effects
formula = "logit_share ~ 1 + price_ + prom_ + C(brand) + C(store)"
model = sm.OLS.from_formula(formula, data=sales_data)
print(model.fit().summary())

                            OLS Regression Results                            
Dep. Variable:            logit_share   R-squared:                       0.510
Model:                            OLS   Adj. R-squared:                  0.509
Method:                 Least Squares   F-statistic:                     476.9
Date:                Tue, 11 Feb 2025   Prob (F-statistic):               0.00
Time:                        18:04:11   Log-Likelihood:                -37268.
No. Observations:               38544   AIC:                         7.471e+04
Df Residuals:                   38459   BIC:                         7.543e+04
Df Model:                          84                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -6.1382      0.045   -1

In [13]:
# question 2 part d (a)
# iv regression  
formula = f"logit_share ~ 1 + prom_ + [price_ ~ cost_]"
iv_wcost_a = IV2SLS.from_formula(formula, sales_data).fit()
print("PART D (a): IV (wholesale cost) Regression Results")
print(iv_wcost_a.summary)

PART D (a): IV (wholesale cost) Regression Results
                          IV-2SLS Estimation Summary                          
Dep. Variable:            logit_share   R-squared:                      0.0095
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0095
No. Observations:               38544   F-statistic:                    254.71
Date:                Tue, Feb 11 2025   P-value (F-stat)                0.0000
Time:                        18:15:28   Distribution:                  chi2(2)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept     -7.9476     0.0136    -584.98     0.0000     -7.9742     -7.9210
p

In [ ]:
# question 2 part d (b)
# iv regression with product fixed effects 
otc_brands = pd.get_dummies(sales_data, columns = ["brand"], drop_first=True)
fixed_effects = " + ".join([col for col in otc_brands.columns if col.startswith("brand_")]) 

formula = f"logit_share ~ 1 + prom_ + {fixed_effects} + [price_ ~ cost_]"
iv_wcost_b = IV2SLS.from_formula(formula, otc_brands).fit()
print("PART D (b): IV (wholesale cost) Regression Results")
print(iv_wcost_b.summary)

PART D (b): IV (wholesale cost) Regression Results
                          IV-2SLS Estimation Summary                          
Dep. Variable:            logit_share   R-squared:                      0.4445
Estimator:                    IV-2SLS   Adj. R-squared:                 0.4444
No. Observations:               38544   F-statistic:                 4.319e+04
Date:                Tue, Feb 11 2025   P-value (F-stat)                0.0000
Time:                        17:48:03   Distribution:                 chi2(12)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept     -7.2116     0.0652    -110.56     0.0000     -7.3394     -7.0837
p

In [11]:
# question 2 part d (c)
# iv regression with product fixed effects 
sales_data["brand_store"] = sales_data["brand"].astype(str) + "_" + sales_data["store"].astype(str)
otc_brand_store = pd.get_dummies(sales_data, columns = ["brand_store"], drop_first=True)
fixed_effects = " + ".join([col for col in otc_brand_store.columns if col.startswith("brand_store_")])

formula = f"logit_share ~ 1 + prom_ + {fixed_effects} + [price_ ~ cost_]"
# iv_wcost_c = IV2SLS.from_formula(formula, otc_brand_store).fit()
# print("PART D (c): IV (wholesale cost) Regression Results")
# print(iv_wcost_c.summary)
# NOTE: I am not running this because it was crashing my kernel every time I tried to run it.

'logit_share ~ 1 + prom_ + brand_store_10_101 + brand_store_10_102 + brand_store_10_103 + brand_store_10_104 + brand_store_10_105 + brand_store_10_106 + brand_store_10_107 + brand_store_10_109 + brand_store_10_110 + brand_store_10_111 + brand_store_10_112 + brand_store_10_113 + brand_store_10_114 + brand_store_10_115 + brand_store_10_116 + brand_store_10_117 + brand_store_10_118 + brand_store_10_119 + brand_store_10_12 + brand_store_10_121 + brand_store_10_122 + brand_store_10_123 + brand_store_10_14 + brand_store_10_18 + brand_store_10_2 + brand_store_10_21 + brand_store_10_28 + brand_store_10_32 + brand_store_10_33 + brand_store_10_40 + brand_store_10_44 + brand_store_10_45 + brand_store_10_47 + brand_store_10_48 + brand_store_10_49 + brand_store_10_5 + brand_store_10_50 + brand_store_10_51 + brand_store_10_52 + brand_store_10_53 + brand_store_10_54 + brand_store_10_56 + brand_store_10_59 + brand_store_10_62 + brand_store_10_64 + brand_store_10_67 + brand_store_10_68 + brand_store_10

In [ ]:
#PART E (a)
# estimate a, b, c with Hausman instrument (average price in other markets)
#---------------------------------------------------------------------------#

# Define Hausman instrument (average price in other markets)
# hausman_iv = sales_data.groupby(["week", "store"])["price_"].transform("mean")
# hausman_iv = sm.add_constant(hausman_iv)

# part a with hausman instrument 
# formula = f"logit_share ~ 1 + prom_ + [price_ ~ hausman_iv]"
# iv_hi_a = IV2SLS.from_formula(formula, sales_data).fit()

#print("PART E (a): Hausman IV Regression Results")
#print(iv_hi_a.summary())

ValueError: Error encountered while checking for nulls in `hausman_iv`: No implementation of `find_nulls()` for type `<class 'formulaic.materializers.types.factor_values.FactorValues'>`.

In [ ]:
#PART E (b)
# estimate a, b, c with Hausman instrument (average price in other markets)
#---------------------------------------------------------------------------#
# part b with hausman instrument with product fixed effects 
# otc_brands = pd.get_dummies(sales_data, columns = ["brand"], drop_first=True)
# fixed_effects = " + ".join([col for col in otc_brands.columns if col.startswith("brand_")]) 

# formula = f"logit_share ~ 1 + prom_ + [price_ ~ hausman_iv] + {fixed_effects}"
# iv_hi_a = IV2SLS.from_formula(formula, sales_data).fit()

# print("\n### PART E (b): Hausman IV Regression with Product Fixed Effects ###")
# print(iv_hi_b.summary())


### PART E (b): Hausman IV Regression with Product Fixed Effects ###
                          IV2SLS Regression Results                           
Dep. Variable:            logit_share   R-squared:                   -5711.116
Model:                         IV2SLS   Adj. R-squared:              -5712.895
Method:                     Two Stage   F-statistic:                    0.8832
                        Least Squares   Prob (F-statistic):              0.564
Date:                Tue, 11 Feb 2025                                         
Time:                        17:00:50                                         
No. Observations:               38544                                         
Df Residuals:                   38531                                         
Df Model:                          12                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------

/usr/local/python/3.12.1/lib/python3.12/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 12, but rank is 3
  warnings.warn('covariance of constraints does not have full '


: 

In [ ]:
#PART E (c)
#NOTE: I am not running this because it was crashing my kernel every time I tried to run it.

# estimate a, b, c with Hausman instrument (average price in other markets)
#---------------------------------------------------------------------------#
# part c with hausman instrument with product-store fixed effects 
#sales_data["brand_store"] = sales_data["brand"].astype(str) + "_" + sales_data["store"].astype(str)
#otc_brand_store = pd.get_dummies(sales_data, columns = ["brand_store"], drop_first=True)
#fixed_effects = " + ".join([col for col in otc_brand_store.columns if col.startswith("brand_store_")])

#formula = f"logit_share ~ 1 + prom_ + [price_ ~ hausman_iv] + {fixed_effects}"
#iv_hi_c = IV2SLS.from_formula(formula, sales_data).fit()

#print("\n### PART E (c): Hausman IV Regression with Product-Store Fixed Effects ###")
#print(iv_hi_c.summary())

In [17]:
# Question 2, part G
# use the analytic formula for elasticity of the logit model to compute the mean own price elasticity of demand for each product in the market 
# using the estimates from part a, b, and c.
# own price elasticity = alpha * price * (1 - market share)
alpha_a = iv_wcost_a.params["price_"]
sales_data["elasticity_a"] = alpha_a * sales_data["price_"] * (1 - sales_data["market_share"])
mean_elasticity_a = sales_data.groupby("brand")["elasticity_a"].mean()

print("Mean Own-Price Elasticities (a)")
print(mean_elasticity_a)

Mean Own-Price Elasticities (a)
brand
1    -0.036319
2    -0.052466
3    -0.074508
4    -0.031472
5    -0.054650
6    -0.086691
7    -0.028398
8    -0.038324
9    -0.042132
10   -0.020482
11   -0.047239
Name: elasticity_a, dtype: float64


In [18]:
# part G (b)
alpha_b = iv_wcost_b.params["price_"]
sales_data["elasticity_b"] = alpha_b * sales_data["price_"] * (1 - sales_data["market_share"])
mean_elasticity_b = sales_data.groupby("brand")["elasticity_b"].mean()

print("Mean Own-Price Elasticities (b) ")
print(mean_elasticity_b)

Mean Own-Price Elasticities (b) 
brand
1    -0.031011
2    -0.044798
3    -0.063619
4    -0.026873
5    -0.046663
6    -0.074021
7    -0.024248
8    -0.032723
9    -0.035975
10   -0.017489
11   -0.040335
Name: elasticity_b, dtype: float64


In [19]:
# part G (c)
alpha_c = iv_wcost_c.params["price_"]
sales_data["elasticity_c"] = alpha_c * sales_data["price_"] * (1 - sales_data["market_share"])
mean_elasticity_c = sales_data.groupby("brand")["elasticity_c"].mean()

print("Mean Own-Price Elasticities (c) ")
print(mean_elasticity_c)

Mean Own-Price Elasticities (c) 
brand
1    -0.121309
2    -0.175243
3    -0.248867
4    -0.105122
5    -0.182537
6    -0.289558
7    -0.094854
8    -0.128007
9    -0.140728
10   -0.068414
11   -0.157784
Name: elasticity_c, dtype: float64


In [59]:
# QUESTION 3
# Random Coefficients Logit Demand + BLP 
# formula for demand model: u_ijt = X_jt * beta + betaB_i * 1{branded product} + alpha_i * p_jt + xsi_jt + epsilon_ijt
# i need to use the demographic data too 
print("Sales Data Columns:", sales_data.columns)
print("Demographic Data Columns:", demo_data.columns)

# Rename the column in the sales_data DataFrame
sales_data.rename(columns={"price_": "prices"}, inplace=True)


Sales Data Columns: Index(['store', 'week', 'brand', 'sales_', 'count', 'prices', 'prom_', 'cost_',
       'total_sales', 'market_share', 'outside_share', 'logit_share',
       'brand_store', 'market_ids'],
      dtype='object')
Demographic Data Columns: Index(['store', 'educ', 'income'], dtype='object')


In [ ]:
# adding in instruments
# List of price instruments for other stores
price_instruments = [f"pricestore{i}" for i in range(1, 31)]  # pricestore1 to pricestore30
# List of instruments (costs + price instruments)
instruments = ["cost_"] + price_instruments

# List of price instruments for other stores
price_instruments = [f"pricestore{i}" for i in range(1, 31)]  # pricestore1 to pricestore30

# List of instruments (costs + price instruments)
instruments = ["cost_"] + price_instruments

In [99]:
# part a 
# estimate the parameters of the mode (beta, alpha, sigma_b, sigman_i) using the BLP estimator 
# as instruments, use both cost and the prices of the same product in the same week at other stores, NOT just an average but a variable for each price at another store in the same period. 
# choose 30 other stores 

#for BLP, i need to define the market which i want to be a store/week combination
# Define the market
sales_data["market_ids"] = sales_data["store"].astype(str) + "_" + sales_data["week"].astype(str)

#Rename the DataFrame in accordance with the BLP requirements
sales_data.rename(columns={"price_": "prices"}, inplace=True)
product_data = sales_data.copy()
product_data.rename(columns={"market_share": "shares"}, inplace=True)
product_data.rename(columns={"brand": "product"}, inplace=True)

#merge the demographic data with the product data based on store 
product_data = product_data.merge(demo_data, on="store")

# i need to create data frame, agent data including market_id, educ, income, store, week
agent_data = product_data[["market_ids", "educ", "income", "store",]].reset_index()

#create a column called PI, which is price * income 
# product_data["PI"] = product_data["prices"] * product_data["income"]

In [111]:
mc_integration = pyblp.Integration('monte_carlo', size=1, specification_options= {'seed': 0})

In [108]:
# add these things in to the agent data
agent_data['nodes0'] = 0
agent_data['nodes1'] = 0
agent_data['weights'] = 1
agent_data

,index,market_ids,educ,income,store,nodes,weights,nodes0,nodes1
0,0,2_1,0.248935,10.55321,2,2,1,0,0
1,1,2_2,0.248935,10.55321,2,2,1,0,0
2,2,2_3,0.248935,10.55321,2,2,1,0,0
3,3,2_4,0.248935,10.55321,2,2,1,0,0
4,4,2_5,0.248935,10.55321,2,2,1,0,0
...,...,...,...,...,...,...,...,...,...
38539,38539,123_44,0.153192,10.33410,123,2,1,0,0
38540,38540,123_45,0.153192,10.33410,123,2,1,0,0
38541,38541,123_46,0.153192,10.33410,123,2,1,0,0
38542,38542,123_47,0.153192,10.33410,123,2,1,0,0


In [102]:
# product formulation and agent formulation
X1_formulation = pyblp.Formulation('0 + prices + prom_', absorb='C(product)')
X2_formulation = pyblp.Formulation('1 + prices')

product_formulations = (X1_formulation, X2_formulation)
product_formulations
agent_formulation = pyblp.Formulation('0 + income + educ')
agent_formulation

income + educ

In [112]:
meds_problem = pyblp.Problem(
    product_formulations,
    product_data,
    agent_formulation,
    agent_data,
    integration=mc_integration,
)
meds_problem

Dimensions:
 T      N     I     K1    K2    D    MD    ED 
----  -----  ----  ----  ----  ---  ----  ----
3504  38544  3504   2     2     2    1     1  

Formulations:
       Column Indices:           0       1   
-----------------------------  ------  ------
 X1: Linear Characteristics    prices  prom_ 
X2: Nonlinear Characteristics    1     prices
       d: Demographics         income   educ 

In [114]:
initial_sigma = np.eye(2)
initial_pi = np.zeros((2, 2))
meds_results = meds_problem.solve(initial_sigma, initial_pi)
print(meds_results)

The model may be under-identified. The total number of unfixed parameters is 4, which is more than the total number of moments, 1. Consider checking whether instruments were properly specified when initializing the problem, and whether parameters were properly configured when solving the problem.


Problem Results Summary:
GMM   Objective    Projected    Reduced Hessian  Reduced Hessian  Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Gradient Norm  Min Eigenvalue   Max Eigenvalue   Shares   Condition Number  Condition Number 
----  ---------  -------------  ---------------  ---------------  -------  ----------------  -----------------
 2    +3.1E-26     +2.3E-13        -1.6E-07         +3.2E-08         0         +1.0E+00          +1.2E+18     

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective   Fixed Point  Contraction
   Time      Converged   Iterations   Evaluations  Iterations   Evaluations
-----------  ---------  ------------  -----------  -----------  -----------
 00:01:00       Yes          0             4          14440        51536   

Nonlinear Coefficient Estimates (Robust SEs in Parentheses):
Sigma:      1         prices    |   Pi:     income     educ  
------  ----------  ----------  |  ------  --------  --------
  1      +1.0E+00